In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
#!pip install langchain sentence-transformers faiss-cpu pypdf transformers torch langchain-community  #instalujemy potrzebne biblioteki

## Wstęp

Na poprzednich zajęciach dowiedzieliście się czym są sieci neuronowe, jak one działają i co można robić za ich pomocą.

W tej lekcji nauczycie się przetwarzać język - czyli zameniać język w postać zrozumiałą dla modeli językowych. Przejdziemy przez proces tokenizacji, tworzenia osadzeń i pipelineów do używania gotowych modeli przez interfejsy API i nie tylko.

Wykład jest bezpośrednią kontynuacją uczenia głębokiego, jednak odbijemy od standardowych technik sieci neuronowych i przyjrzymy się nowoczesnym architekturom przetwarzania języka naturalnego - transformerom.

Zobaczycie także jak tworzy się nowoczesne aplikacje z wykorzystaniem modeli NLP.

Co znajduje się w notatniku ?

Najważniejsze informacje do pracy z transformerami, ich struktura, zasada działania, mechanizmy uwagi i wiele ciekawych rzeczy, które dały podstawy do stworzenia dużych modeli językowych (GPT, DeepSeek, Claude itp.).

Znajdziecie tu także snippety kodu, które pozwolą wam zrealizować projekt z tych zajęć z wykrozystaniem biblioteki HuggingFace 🤗 oraz Langchain 🦜🔗

Podobnie jak w ostatnim notebooku, w notebooku napotkasz następujące symbole:

- ❓ dodatkowe zagadnienie do samodzielnej realizacji, które pozwoli ci jeszcze bardziej poszerzyć wiedzę,
- 💡 wprowadza ważne, prawdopodobnie nieznane ci narzędzie w Pythonie lub pojęcie, które warto zapamiętać,
- ☝ wskazuje na miejsca potencjalnych błędów czy pomyłek.

Dodatkowo, jeśli chcielibyście dowiedzieć się więcej o LLMach, modelach NLP i biliotece HuggingFace 🤗, odwiedźcie oficjalny [poradnik od zespołu HF](https://huggingface.co/learn/llm-course/chapter0/1)





## 1. Tokenizacja
### **Tokenizacja – co to jest?**  
🔹 **Proces segmentacji tekstu** – polega na podziale ciągów znaków (zdań, wyrażeń) na mniejsze jednostki zwane **tokenami**.  
🔹 **Podstawa przetwarzania języka** – stosowana w NLP (przetwarzaniu języka naturalnego), analizie tekstu i modelach AI.  
🔹 **Token ≠ słowo** – tokenem może być:  
   • pojedynczy wyraz (np. *"kot"*),  
   • znak interpunkcyjny (np. *","*),  
   • liczba (np. *"42"*),  
   • fragment wyrazu (w językach bez spacji, np. *"深度学习" → "深度", "学习"*).  

### **Dlaczego tokenizacja jest ważna?**  
✅ **Uproszczenie analizy** – komputery lepiej rozumieją małe, ustandaryzowane fragmenty niż całe zdania.  
✅ **Przygotowanie danych** – niezbędny krok przed wektoryzacją (np. dla modeli typu BERT, GPT).  
✅ **Elastyczność** – różne metody tokenizacji (np. BPE, WordPiece) dostosowane do różnych języków i zadań.  

### **Przykłady tokenizacji**  
📌 *"Ala ma kota."* → `["Ala", "ma", "kota", "."]` (tokeny pojedyncze)  
📌 *"I'm happy!"* → `["I", "'", "m", "happy", "!"]` (rozbicie skrótów)  
📌 *"今日は晴れです"* → `["今日", "は", "晴れ", "です"]` (tokeny w japońskim)  

### **Wyzwania w tokenizacji**  
⚠️ **Języki bez spacji** (np. chiński) – wymagają zaawansowanych algorytmów.  
⚠️ **Wieloznaczność** (np. *"3.14"* – liczba czy zdanie z kropką?).  
⚠️ **Skróty i slang** (np. *"działa xd"* → czy *"xd"* to osobny token?).  

Tokenizacja to **kluczowy krok**, który wpływa na skuteczność modeli językowych.

<image src="https://spacy.io/images/tokenization.svg"></image>  

Za pomocą biblioteki SpaCy możemy łatwo przetworzyć tekst za pomocą gotowych tokenizatorów, aby podzielić tekst na tokeny


In [3]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("Apple is looking at buying U.K. startup for $1 billion")

print([token.text for token in doc])


OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

Jednakże, w tej lekcji będziemy korzystać z gotowych, wyuczonych tokenizerów, które poza dzieleniem tekstu na tokeny przypisują im unikatowe liczby :

- 🔢 Mapowanie tokenów na liczby – każdy token otrzymuje unikalny identyfikator (np. "kot" → 5041, "!" → 999).
- 📚 Słownik tokenów (vocab) – wstępnie zdefiniowany zbiór wszystkich możliwych tokenów i ich numerycznych reprezentacji.
- ⚡ Integracja z modelami NLP – tokenizery (np. BERT, GPT) są dostosowane do architektury konkretnego modelu, co gwarantuje spójność danych wejściowych.

💡 Z pomocą przychodzi biblioteka Transformers od HuggingFace 🤗

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokens = tokenizer.encode("I am tokenizing my first sentence!")
print(tokens)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

[101, 1045, 2572, 19204, 6026, 2026, 2034, 6251, 999, 102]


Trochę to trwało, ale model został pobrany i załadowany - teraz podczas używania go w notatniku, nie jest wymagane jego ponowne pobieranie do momentu restartu.

Jak widzicie pobraliśmy tokenizer, który jest złączony z modelem BERT, o którym powiemy sobie trochę później. Skupmy się na fragmencie **tokens = tokenizer.encode("Ala ma kota.")** - nasz napis zostaje podzielony na 6 tokenów, które są reprezentowane przez liczby.

 Jak teraz sprawdzić jakie to są słowa ? To bardzo proste :

In [ ]:
decoded_tokens = tokenizer.convert_ids_to_tokens(tokens)
print(decoded_tokens)

['[CLS]', 'i', 'am', 'token', '##izing', 'my', 'first', 'sentence', '!', '[SEP]']


Zwróćmy uwagę na trzy rzeczy :

*   tokeny [CLS] i [SEP] wskazują początek i koniec sekwencji - mogą być różne dla różnych modeli (może ich nawet nie być w cale)
*   wszystkie tokeny zostały zamienione na małe litery
* słowo *tokenizing* zostało podzielone na 2 tokeny, ponieważ lepiej użyć dwóch znanych tokenów niż dodawać miliony rzadkich słów do słownika.

To na razie tyle, jeśli chodzi o tokenizację. Przejdziemy teraz do bardziej interesującego zagadnienia jakim są transformery ✈



# 2. Transformery

📌 Czym są transformery?

Nowoczesna architektura sieci neuronowych wprowadzona w 2017 roku w pracy ["Attention Is All You Need"](https://arxiv.org/abs/1706.03762), która zrewolucjonizowała NLP.

![Transformer Architecture](https://d2l.ai/_images/transformer.svg)

🔧 Architektura Encoder-Decoder
- **Encoder** | Analizuje wejściowy tekst | Tłumaczenie: analiza zdania źródłowego
- **Decoder** | Generuje wynik | Tłumaczenie: tworzenie zdania docelowego


🧩 Jak to działa w praktyce?
1. **Tokenizacja** - dzielenie tekstu na części (np. słowa/subwordy)
2. **Embedding** - zamiana na wektory liczbowe
3. **Przetwarzanie** przez warstwy transformerów
4. **Generacja** wyniku

📚 Najpopularniejsze Modele
1. **BERT** - do analizy tekstu
2. **GPT** - do generacji tekstu
3. **T5** - do wielozadaniowości

🌟 Dlaczego warto używać?
- ✔️ Lepsze wyniki niż RNN/LSTM  
- ✔️ Możliwość przetwarzania równoległego  
- ✔️ Bogate pretrenowane modele dostępne od ręki

❓ Więcej na temat transformerów znajdziecie w kursie ["Attention Mechanisms and Transformers"](https://d2l.ai/chapter_attention-mechanisms-and-transformers/index.html)



# 4. Duże modele językowe (LLM)

### Podstawowa charakterystyka

Duże modele językowe (LLM) to zaawansowane systemy AI oparte na architekturze transformerów, zdolne do przetwarzania i generowania tekstu na poziomie zbliżonym do ludzkiego. Ich kluczową cechą jest skalowalność - najnowsze modele mają setki miliardów parametrów i są trenowane na petabajtach danych tekstowych.

### Kluczowe elementy:
- **Transformer architecture**: rdzeń wszystkich współczesnych LLM
- **Skala parametrów**: od kilku milionów do bilionów
- **Zastosowania**: chatboty, generowanie kodu, tłumaczenia, kreatywne pisanie

### Zastosowania i ograniczenia

**Główne zastosowania:**
- Automatyzacja tworzenia treści
- Zaawansowane systemy Q&A
- Personalizowane nauczanie
- Generowanie i debugowanie kodu

**Poważne wyzwania:**
- Ogromne koszty trenowania ($$$ i ślad węglowy)
- Problem "halucynacji" (generowanie nieprawdziwych informacji)
- Wrażliwość na prompt engineering
- Kwestie etyczne (bias, dezinformacja)

## Przyszłość LLM
Nadchodząca ewolucja wskazuje na:
- Lepsze zarządzanie kontekstem (dłuższe okna)
- Integracja multimodalna (tekst+obraz+dźwięk)
- Specializowane wersje dla konkretnych domen
- Optymalizacja pod kątem efektywności energetycznej
- Agentyzacja modeli

Ze względu na skalę wieloparametrowych LLMów takich jak GPT4 czy DeepSeek R1, nie są one możliwe do trenowania na lokalnych maszynach - potrzebują ogromnych stacji obliczeniowych, złożonych głównie z procesorów graficznych GPU.

❓Jednakże istnieje możliwość korzystania z nich za pomocą płatnych interfejsów API lub hostowania modeli lokalnie np. za pomocą oprogramowania [Ollama](https://ollama.com/)

Więcej o użyciu dużych modeli językowych, w rozdziale poświęconym bibliotece Langchain 🦜🔗

# 3. Biblioteka Hugging Face 🤗

W tym kursie nie będziemy zagłębiać się w implementację modeli transformerowych - skorzystamy z gotowców, które dostarcza biblioteka Hugging Face 🤗

In [ ]:
from transformers import pipeline, set_seed

# Inicjalizacja generatora tekstu
generator = pipeline('text-generation', model='gpt2')

# Opcjonalnie: ustawienie ziarna dla powtarzalności wyników
# set_seed(42)

# Generowanie tekstu
result = generator(
    "In today world, AI is",  # Początek tekstu
    max_length=100,          # Maksymalna długość wygenerowanego tekstu
    num_return_sequences=1,  # Liczba wariantów do wygenerowania
    do_sample=True,         # Czy stosować próbkowanie
    top_k=30,               # Ograniczenie do top 30 najbardziej prawdopodobnych słów
    temperature=0.7,        # Kontrola kreatywności (0.7 = średnia kreatywność)
)

# Wyświetlenie wyniku
print(result[0]['generated_text'])

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

Taki prosty kod pozwala nam na uzycie modelu GPT-2 od OpenAI, który odpowiada za generowanie tekstu (głównie w języku angielskim). Jak możecie zauważyć, nie jest to wysokiej klasy teskt, ale dużym przełowem w rozwoju NLP było to, że matematyczny model potrafił generować tekst, który potrafi zrozumieć człowiek!

Pewnie zastanawiacie się gdzie jest tokenizer, o którym była mowa wcześniej ? Odpowiedź jest bardzo prosta - jest on wbudowany w funckję **pipeline**, która zapewnia przetworzenie potokowe zdania w języku angielskim.

![Pipeline](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/full_nlp_pipeline-dark.svg)

Więcej o pipeline'ach i ich użyciach znajdziecie w oficjalnej dokumentacji [Pipeline](https://huggingface.co/docs/transformers/main_classes/pipelines)

Poza tym Hugging Face 🤗 oferuje wiele przeróżnych narzędzi, które możecie wykorzystywać w codzinnej pracy z modelami produkcyjnymi, więc jest to niezbędne narzędzie dla specjalisty od AI ! Warto zobaczyć również bibliotekę [Datasets](https://huggingface.co/datasets), która zawiera sprawdzone dane do trenowania nowych modeli lub finetune'owania już stworzonych

# Langchain 🦜🔗

**LangChain** to framework do tworzenia aplikacji wykorzystujących modele językowe (LLM), takie jak GPT. Umożliwia łączenie LLM z zewnętrznymi źródłami danych, narzędziami i pamięcią, co ułatwia budowanie zaawansowanych systemów AI.  

### **Kluczowe funkcje:**  
🔹 **Integracja z modelami** (OpenAI, Hugging Face itp.)  
🔹 **Łańcuchy (Chains)** – łączenie wielu operacji w przepływy  
🔹 **Zarządzanie pamięcią** (np. zapamiętywanie kontekstu w rozmowie)  
🔹 **Praca z danymi** (PDF, bazy danych, internet)  
🔹 **Agenci** – dynamiczne wywoływanie narzędzi w zależności od danych  

### **Zastosowania:**  
- Chatboty z dostępem do własnych danych  
- Automatyzacja zadań z użyciem AI  
- Analiza dokumentów i generowanie treści  

LangChain jest dostępny w językach **Python** i **JavaScript**.  

Jest to bardzo popularna biblioteka do tworzenia nowowczesnych aplikacji z wykrozystaniem LLMów, a także wszelakich modeli NLP lub nawet modeli wizyjnych.

❓Dodatkowo istnieją biblioteki współtowarzyszące procesowi tworzenia aplikacji w LangChain, są to :
- LangGraph 🌐 - biblioteka do tworzenia grafowych sekwencji przepływu programu i sterowanie nim
- LangSmith 🛠️ - biblioteka do monitorowania apliakcji z grupy LangChain, umożliwiając min. podgląd generowanych treści, wydajności systemu czy zgłaszanych błędów produkcyjnych

Podsumowując : 💡 LangChain + LangGraph + LangSmith = Potężny stack do budowy zaawansowanych systemów AI

Okej, mając teorię za sobą, przechodzimy do praktycznego użycia biblioteki Langchain 🦜🔗 ! Znajdziecie tutaj wszystkie snippety potrzebne do wykonania zadania związanego z lekcją - systemu uzyskiwania odzyskiwania rozszerzonej generacji - RAG (Retrieval Augmented Generation)

![RAG](https://www.promptingguide.ai/_next/image?url=%2F_next%2Fstatic%2Fmedia%2Frag-framework.81dc2cdc.png&w=3840&q=75)

### Ładowanie i przetwarzanie dokumentu (PDF)
[Document loaders](https://python.langchain.com/docs/integrations/document_loaders/) - inne loadery do dokumentów, znajdziecie tam wczytywanie i splittowanie plików .csv, .txt i wiele wiele innych

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Ładowanie pliku PDF
loader = PyPDFLoader("/content/language_understanding_paper.pdf")
pages = loader.load()

# Podział tekstu na fragmenty
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=100,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)
docs = text_splitter.split_documents(pages)

for idx,doc in enumerate(docs):
  print(f"[{idx}].{doc}\n\n")

[0].page_content='Improving Language Understanding
by Generative Pre-Training
Alec Radford
OpenAI
alec@openai.com
Karthik Narasimhan
OpenAI
karthikn@openai.com
Tim Salimans
OpenAI
tim@openai.com
Ilya Sutskever
OpenAI
ilyasu@openai.com
Abstract' metadata={'producer': 'pdfTeX-1.40.18', 'creator': 'LaTeX with hyperref package', 'creationdate': '2018-06-08T19:14:34+00:00', 'author': '', 'keywords': '', 'moddate': '2018-06-08T19:14:34+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017) kpathsea version 6.2.3', 'subject': '', 'title': '', 'trapped': '/False', 'source': '/content/language_understanding_paper.pdf', 'total_pages': 12, 'page': 0, 'page_label': '1'}


[1].page_content='Tim Salimans
OpenAI
tim@openai.com
Ilya Sutskever
OpenAI
ilyasu@openai.com
Abstract
Natural language understanding comprises a wide range of diverse tasks such
as textual entailment, question answering, semantic similarity assessment, and' metadata={'producer': 'pdfTeX-1.40.18

W ten sposób nasz PDF został poszatkowany na części w prosty sposób i jest gotowy do przetworzenia dalej - będzie to podstawa informacyjna dla naszego modelu.

###  Tworzenie wektorowej bazy danych z użyciem Hugging Face Embeddings

### **Tworzenie wektorowej bazy danych z Hugging Face Embeddings**  

#### **Co to jest wektorowa baza danych?**  
Wektorowa baza danych (np. **FAISS** w naszym przykładzie) to specjalny rodzaj bazy, która przechowuje dane jako **wektory liczbowe** (tzw. *embeddings*). Każdy fragment tekstu (np. zdanie, akapit) jest zamieniany na ciąg liczb reprezentujących jego znaczenie.  

- **Działa jak wyszukiwarka semantyczna** – znajduje fragmenty podobne znaczeniowo do zapytania (a nie tylko dosłowne dopasowania).  
- **Optymalizuje szybkie wyszukiwanie** – dzięki indeksowaniu wektorów (FAISS jest wydajną biblioteką od Meta).  

#### **Jak to działa w naszym kodzie?**  
1. **Generujemy embeddings** (numeryczną reprezentację tekstu):  
   ```python
   embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
   ```  
   - Używamy modelu `all-mpnet-base-v2` z Hugging Face, który zamienia tekst na wektory 768 liczb.  
   - Model jest wytrenowany do rozumienia znaczenia zdań (np. "koty lubią mleko" i "mleko jest ulubionym napojem kotów" będą miały podobne wektory).  

2. **Tworzymy bazę FAISS**:  
   ```python
   db = FAISS.from_documents(docs, embeddings)
   ```  
   - Dla każdego fragmentu dokumentu (`docs`) obliczamy embedding i zapisujemy go w bazie.  
   - FAISS indeksuje wektory, aby później szybko wyszukiwać najbliższe znaczeniowo wyniki.  

3. **Zapisujemy bazę do pliku** (opcjonalnie):  
   ```python
   db.save_local("faiss_index_hf")
   ```  
   - Możesz ją później wczytać bez ponownego przetwarzania dokumentów.  

#### **Przykład wyszukiwania**  
Gdy zadamy pytanie np. "Jakie są główne wnioski z dokumentu?":  
1. Zapytanie też jest zamieniane na embedding.  
2. FAISS porównuje go z wektorami w bazie i zwraca **najbardziej podobne fragmenty tekstu**.  

#### **Dlaczego to ważne w RAG?**  
- **Retrieval (pobieranie)** – baza wektorowa znajduje fragmenty dokumentu związane z pytaniem.  
- **Generation (generowanie)** – model LLM (np. Zephyr) dostaje te fragmenty jako kontekst i generuje odpowiedź.  

#### **💡 Pamiętajcie:**  
- Im lepszy model embeddingów, tym trafniejsze wyniki wyszukiwania.  
- Możecie testować inne modele z [Hugging Face](https://huggingface.co/models?pipeline_tag=sentence-similarity&sort=trendingl).  

---

W następnym kroku wykorzystujemy tę bazę do **RAG** – model LLM dostaje znalezione fragmenty i generuje odpowiedź!

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Model embeddings z Hugging Face
model_name = "sentence-transformers/all-mpnet-base-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

# Tworzenie i zapis bazy FAISS
db = FAISS.from_documents(docs, embeddings)
db.save_local("faiss_index_hf")

/tmp/ipython-input-1253801366.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Baza powinna utworzyć się w folderze **faiss_index_hf** - sprawdź pokrótce jej zawartość i lecimy dalej !

## Wyszukiwanie podobnych fragmentów (Retrieval)

In [ ]:
# Wczytanie bazy danych
db = FAISS.load_local("faiss_index_hf", embeddings, allow_dangerous_deserialization=True)

# Wyszukiwanie podobnych fragmentów
query = "What is the usage of semi-supervised learning for NLP in article?"
similar_docs = db.similarity_search(query, k=3)  # top 3 wyniki

for doc in similar_docs:
    print(doc.page_content[:300] + "...\n---")

form of semi-supervised learning. Early work by Collobert and Weston [10] used a wide variety of
auxiliary NLP tasks such as POS tagging, chunking, named entity recognition, and language modeling
to improve semantic role labeling. More recently, Rei [50] added an auxiliary language modeling...
---
knowledge for downstream tasks.
2 Related Work
Semi-supervised learning for NLP Our work broadly falls under the category of semi-supervised
learning for natural language. This paradigm has attracted signiﬁcant interest, with applications to...
---
it difﬁcult to develop effective semi-supervised learning approaches for language processing.
1https://gluebenchmark.com/leaderboard
Preprint. Work in progress....
---


Nie jest to idealne rozwiązanie do uzyskiwania podobnych wyszukań, ale jest ono zadowalające i wykorzystywane w wielu systemach, gdzie nie można pozwolić sobie na wtałaczanie ogromnych ilości informacji - czy to przez ograniczenia sprzętowe lub okno kontekstowe modelu.

❓ Możemy również wykorzystywać inne systemy takie jak relacyjne i nierelacyjne bazy danych (SQL, NoSQL) lub grafowe bazy danych (Neo4j)

## Generowanie odpowiedzi z wykorzystaniem lokalnego modelu LLM (Hugging Face)

Korzystamy tu z wcześniej poznanej technologii Pipelineów i wykorzystujemy model [Flan-T5](https://huggingface.co/google/flan-t5-small), który był pierwszym z modeli NLP, który rozwiązywał wszystkie problemy przetwarzania języka : podsumowywanie, generowanie tesktu czy właśnie odpowiadanie na pytania (QA)

Wykorzystujemy tu także gotowy łańcuch do pobierania danych i wtłaczania informacji do okna kontekstowego modelu - [RetrievalQA](https://python.langchain.com/api_reference/langchain/chains/langchain.chains.retrieval_qa.base.RetrievalQA.html)

In [ ]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
from langchain.chains import RetrievalQA

model_name = "google/flan-t5-small"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=1024
)

llm = HuggingFacePipeline(pipeline=pipe)

# Stworzenie łańcucha RAG
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever()
)

result = qa_chain.invoke("What is the usage of semi-supervised learning for NLP in article?")
print(f"AI Assistant: {result['result']}")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu
/tmp/ipython-input-239172684.py:16: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


AI Assistant: POS tagging, chunking, named entity recognition, and language modeling to improve semantic role labeling


W ten oto sposób zbudowaliśmy pierwszego asystenta AI, który będzie odpowiadał na zadawane przez was pytania dotyczące artykułu naukowego.

# Podsumowanie

W tym module poznaliście kluczowe narzędzia NLP – od tokenizacji i transformerów, przez pracę z Hugging Face i LangChain, aż po tworzenie wektorowych baz danych i generowanie odpowiedzi z LLM. Poprzez zadanie dołączone do lekcji przetrenujecie praktyczn wdrażania tych technologii, zaczynając od przetwarzania dokumentów PDF, a kończąc na budowaniu systemów RAG.

(W razie pytań – jesteśmy tu, by pomóc!) 😊

Dodatowo korzystajcie z ogólnodostępnych implementacji, dokumentacji i oczywiście pomocy LLMów w postaci chatów, ale pamiętajcie - dla własnego dobra i zasad fair play, róbcie to z głową i korzystajcie mądrze z narzędzi AI.

🔗 Przydatne linki:

[Dokumentacja Hugging Face](https://huggingface.co/docs)

[Dokumentacja LangChain](https://python.langchain.com/docs/get_started/introduction)

[Modele Hugging Face](https://huggingface.co/models)

[Zalecenia do korzystania z AI](https://di.pwr.edu.pl/oprogramowanie/ai/zalecenia-korzystania-z-narzedzi-ai)